<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/LSTM_squared_June14%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#I want to integrate a language model and an emotion classifier

In [1]:
#install depdencies

!pip install keras tensorflow scikit-learn numpy

In [2]:
#Import dependencies

import tensorflow as tf
from tensorflow import keras

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import pickle

In [3]:
#Creating afile to hold the path

from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


#Load Dataset and Tokenizer

In [4]:
# Load dataset
path = "/content/drive/MyDrive/Sean/Chatbot_May,2023/Inference_Fear_Gut_100books,2023.csv"
df = pd.read_csv(path)

# Load tokenizer
tokenizer_path = '/content/drive/MyDrive/Sean/Emoclass/Tokenizer_Universal_125Kvoc_May5.2023.pickle'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

In [5]:
df.head()

,Unnamed: 0,text,emotion
0,0,ballads by horatio alger contents ballads bald...,calm
1,1,harvard odes i fair harvard dear guide of our ...,calm
2,2,in the days of whig and tory under royal georg...,calm
3,3,while beneath are stalled the cattle with thei...,calm
4,4,daughters growing up from childish plays one b...,calm


In [6]:
df.shape

(79125, 3)

#Defining Emotion Axis

In [ ]:
emotion_dict = {'dread': 0, 'fear': 1, 'anxiety': 2, 'worry': 3, 'concern': 4, 'calm': 5}

#Processing Text

1) Needs to be changed to accomodate new input dataset
2) Add an embedding layer...


In [ ]:
import numpy as np
import pandas as pd

from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import tensorflow as tf
import pickle

# Load dataset
path = "/content/drive/MyDrive/Sean/Chatbot_May,2023/Inference_Fear_Gut_100books,2023.csv"
df = pd.read_csv(path)

# Load tokenizer
tokenizer_path = '/content/drive/MyDrive/Sean/Emoclass/Tokenizer_Universal_125Kvoc_May5.2023.pickle'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Define parameters
sequence_length = 75
word_embedding_dim = 100
emotion_embedding_dim = 10
emotion_dict = {'dread': 0, 'fear': 1, 'anxiety': 2, 'worry': 3, 'concern': 4, 'calm': 5}

# Initialize lists to hold X and y
X = []
X_emotion = []
y = []

# Iterate over each text and emotion in the dataframe
for text, emotion in zip(df['text'], df['emotion']):
    # Convert text to a sequence of tokens
    sequence = tokenizer.texts_to_sequences([text])[0]
    emotion_index = emotion_dict[emotion]

    # Slide a window across the sequence
    for i in range(sequence_length, len(sequence)):
        # Add the window to X and the corresponding emotion to X_emotion
        X.append(sequence[i-sequence_length:i])
        X_emotion.append([emotion_index]*sequence_length)
        y.append(sequence[i])

# Convert X and y to numpy arrays
X = np.array(X)
X_emotion = np.array(X_emotion)
y = np.array(y)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Generate word embeddings
word_embedding_layer = Embedding(input_dim=vocab_size, output_dim=word_embedding_dim, input_length=sequence_length)
word_embeddings = word_embedding_layer(X)

# Generate emotion embeddings
emotion_embedding_layer = Embedding(input_dim=len(emotion_dict), output_dim=emotion_embedding_dim, input_length=sequence_length)
emotion_embeddings = emotion_embedding_layer(X_emotion)

# Concatenate word and emotion embeddings
concatenated_embeddings = tf.keras.layers.concatenate([word_embeddings, emotion_embeddings], axis=-1)

# Specify the model
model = Sequential()
model.add(LSTM(256, input_shape=(sequence_length, word_embedding_dim + emotion_embedding_dim)))
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(concatenated_embeddings, np.expand_dims(y, -1), epochs=10, batch_size=16)


Epoch 1/10
23420/24697 [===========================>..] - ETA: 13s - loss: 7.2310

In [7]:
import numpy as np
from keras.preprocessing.text import Tokenizer


# Load tokenizer
tokenizer_path = '/content/drive/MyDrive/Sean/Emoclass/Tokenizer_Universal_125Kvoc_May5.2023.pickle'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Initialize lists to hold X and y
X = []
y = []

# Iterate over each text in the dataframe
for text in df['text']:
    # Convert text to a sequence of tokens
    sequence = tokenizer.texts_to_sequences([text])[0]

    # Slide a window across the sequence
    for i in range(100, len(sequence)):
        # Add the window to X
        X.append(sequence[i-100:i])

        # Add the next word to y
        y.append(sequence[i])

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)


#New way for text processing

In [10]:
from keras.layers import Embedding, concatenate, LSTM, Dense, TimeDistributed
from keras.models import Sequential

import tensorflow as tf
import numpy as np

# Load the tokenizer
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Initialize lists to hold X and y
X = []
y = []

# Iterate over each text and emotion in the dataframe
for text, emotion in zip(df['text'], df['emotion']):
    # Convert text to a sequence of tokens
    sequence = tokenizer.texts_to_sequences([text])[0]
    emotion_index = emotion_dict[emotion]

    # Slide a window across the sequence
    for i in range(75, len(sequence)):
        # Add the window to X and the corresponding word to y
        X.append(sequence[i-75:i])
        y.append(sequence[i])

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

# Generate word embeddings
vocab_size = len(tokenizer.word_index) + 1
word_embedding_dim = 100
word_embedding_layer = Embedding(input_dim=vocab_size, output_dim=word_embedding_dim)
word_embeddings = word_embedding_layer(X)

# Encode emotions to indices and generate emotion embeddings
emotion_dict = {'dread': 0, 'fear': 1, 'anxiety': 2, 'worry': 3, 'concern': 4, 'calm': 5}
emotion_indices = df['emotion'].map(emotion_dict).values
emotion_vocab_size = len(emotion_dict)
emotion_embedding_dim = 10
emotion_embedding_layer = Embedding(input_dim=emotion_vocab_size, output_dim=emotion_embedding_dim)
emotion_embeddings = emotion_embedding_layer(emotion_indices)

# Repeat emotion embeddings for each word in the sequence
emotion_embeddings = tf.keras.backend.expand_dims(emotion_embeddings, 1)
emotion_embeddings = tf.keras.backend.repeat_elements(emotion_embeddings, rep=75, axis=1)

# Concatenate word and emotion embeddings
concatenated_embeddings = np.concatenate([word_embeddings, emotion_embeddings], axis=-1)

# Specify and compile the model
model = Sequential()
model.add(LSTM(256, input_shape=(75, word_embedding_dim + emotion_embedding_dim)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


ValueError: ignored

In [7]:
# Initialize lists to hold X and y
X = []
y = []

# Iterate over each text and emotion in the dataframe
for text, emotion in zip(df['text'], df['emotion']):
    # Convert text to a sequence of tokens
    sequence = tokenizer.texts_to_sequences([text])[0]
    emotion_index = emotion_dict[emotion]

    # Slide a window across the sequence
    for i in range(75, len(sequence)):
        # Add the window to X and the corresponding emotion to X_emotion
        X.append(sequence[i-75:i])
        y.append(sequence[i])

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)


In [6]:

from keras.layers import Embedding, concatenate, LSTM, Dense, TimeDistributed
from keras.models import Sequential
import tensorflow as tf

# Load the tokenizer and generate sequences
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)
sequences = tokenizer.texts_to_sequences(df['text'])
sequences = pad_sequences(sequences, maxlen=75)

# Generate word embeddings
vocab_size = len(tokenizer.word_index) + 1
word_embedding_dim = 100
word_embedding_layer = Embedding(input_dim=vocab_size, output_dim=word_embedding_dim)
word_embeddings = word_embedding_layer(sequences)

# Encode emotions to indices and generate emotion embeddings
emotion_dict = {'dread': 0, 'fear': 1, 'anxiety': 2, 'worry': 3, 'concern': 4, 'calm': 5}
emotion_indices = df['emotion'].map(emotion_dict).values
emotion_vocab_size = len(emotion_dict)
emotion_embedding_dim = 10
emotion_embedding_layer = Embedding(input_dim=emotion_vocab_size, output_dim=emotion_embedding_dim)
emotion_embeddings = emotion_embedding_layer(emotion_indices)

# Repeat emotion embeddings for each word in the sequence
emotion_embeddings = tf.keras.backend.expand_dims(emotion_embeddings, 1)
emotion_embeddings = tf.keras.backend.repeat_elements(emotion_embeddings, rep=75, axis=1)

# Concatenate word and emotion embeddings
concatenated_embeddings = np.concatenate([word_embeddings, emotion_embeddings], axis=-1)

# Specify and compile the model
model = Sequential()
model.add(LSTM(256, input_shape=(75, word_embedding_dim + emotion_embedding_dim)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


In [ ]:
#Train the model

model.fit(concatenated_embeddings, np.expand_dims(y, -1), epochs=10)


In [8]:
model.fit(concatenated_embeddings, np.expand_dims(y, -1), epochs=10, batch_size=16)


ValueError: ignored